In [12]:
import pandas as pd
import holidays
import datetime

In [13]:
df_z = pd.read_excel('2.xlsx', sheet_name='Заявки')
df_h = pd.read_excel('2.xlsx', sheet_name='Холды')

In [14]:
df_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10689 entries, 0 to 10688
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_zayavki  10689 non-null  object        
 1   hold        10689 non-null  object        
 2   hold_start  10689 non-null  datetime64[ns]
 3   hold_end    9582 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 334.2+ KB


In [15]:
df_z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42209 entries, 0 to 42208
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ИД заявки                  42209 non-null  object
 1   Вид оценки                 42209 non-null  object
 2   Сегмент                    42209 non-null  object
 3   подразделение эксперта     42209 non-null  object
 4   Цель рассмотрения заявки   42209 non-null  object
 5   Дата закрытия заявки       42209 non-null  object
 6   Дата открытия заявки       42209 non-null  object
 7   Назначение заявки          42209 non-null  object
 8   Автоматизация заявки       42209 non-null  object
 9   Классификатор Плышевского  42209 non-null  object
dtypes: object(10)
memory usage: 3.2+ MB


In [16]:
df_z[df_z['Дата открытия заявки'] == '-1'].count()

ИД заявки                    1885
Вид оценки                   1885
Сегмент                      1885
подразделение эксперта       1885
Цель рассмотрения заявки     1885
Дата закрытия заявки         1885
Дата открытия заявки         1885
Назначение заявки            1885
Автоматизация заявки         1885
Классификатор Плышевского    1885
dtype: int64

In [17]:
df_z = df_z[df_z['Дата открытия заявки'] !='-1']
df_z['Дата открытия заявки'] = df_z['Дата открытия заявки'].astype('datetime64[ns]')
df_z['Дата закрытия заявки'] = df_z['Дата закрытия заявки'].astype('datetime64[ns]')

В данном случае я заметил,что в некоторых строках в дате открытия стоит -1. Я бы уточнил что это значит и нет ли ошибки,
но так как сейчас такой возможности нет, я исключу эти заявки, темболее что их около 5% от общей массы.

В дальнейшем я буду учитывать -1 как отсутствие данных. Хоть его присутсиве в других столбцах и говорит что это не совсем верно.

Теперь даты это даты,а с доп параметрами можно будет разбираться дальше. По ходу задач.

In [18]:
merged_df = df_h.merge(df_z,left_on='id_zayavki', right_on='ИД заявки', how='left')
df_h['hold_end'] = merged_df['Дата закрытия заявки']
df_h['hold_end'] = df_h['hold_end'].astype('datetime64[ns]')

Я заполнил пропуски в окончании холдов с помощью дат закрытия заявкок

Все закрытия холда где null я сразу заполнил значениями

##### Определить общее время выполнения для каждой заявки как разницу между датами закрытия и открытия заявок (в сутках).

In [19]:
def date_difference(row):
    return row['Дата закрытия заявки'] - row['Дата открытия заявки']

df_z['Длительность'] = df_z.apply(date_difference, axis=1).dt.days

Я сделал это через функцию,которая находит разницу между концом и началом и выделяет из этого дни

##### Определить кол-во рабочих часов, необходимых для выполнения каждой заявки. 

Вот тут ниже небольшое извращение)) Оно рабивает период между началом и концом на секунды и каждую проверяет на вхождение. 
Наверно более быстрым вариантом было бы изначально по периоду создавать ряд дат и каждую проверять на выходные, праздники и тп и из этого
считать количество времени в сутках, которое считается рабочим и после складывать. т.е каждую дату брать за 24 часа и после вычитать не рабочее время и после складывать


In [20]:
# Задаем рабочие часы и исключения
WORK_HOURS = {
    'monday': [(9, 0, 18, 0)],
    'tuesday': [(9, 0, 18, 0)],
    'wednesday': [(9, 0, 18, 0)],
    'thursday': [(9, 0, 18, 0)],
    'friday': [(9, 0, 16, 45)]
}

LUNCH_BREAK = (12, 15, 13, 0)
RU_HOLIDAYS = holidays.RU(years=[2022, 2023, 2024, 2025])

def is_working_time(dt):
    if dt.weekday() > 4 or dt.date() in RU_HOLIDAYS:
        return False
    day_name = dt.strftime('%A').lower()
    for start_h, start_m, end_h, end_m in WORK_HOURS.get(day_name, []):
        start_time = datetime.datetime(dt.year, dt.month, dt.day, start_h, start_m)
        end_time = datetime.datetime(dt.year, dt.month, dt.day, end_h, end_m)
        if start_time <= dt < end_time:
            return not (datetime.datetime(dt.year, dt.month, dt.day, *LUNCH_BREAK[:2]) <= dt < datetime.datetime(dt.year, dt.month, dt.day, *LUNCH_BREAK[2:]))
    return False

def calculate_working_seconds(start_dt, end_dt):
    current_dt = start_dt
    working_seconds = 0
    while current_dt < end_dt:
        if is_working_time(current_dt):
            working_seconds += 60  # Прибавляем минуту, если это рабочее время
        current_dt += datetime.timedelta(minutes=1)
    return working_seconds


# Вычисляем рабочие часы для каждой строки в DataFrame. Секунды делим на 60 на 60 и на 8
df_z['Рабочие дни'] = df_z.apply(lambda row: calculate_working_seconds(row['Дата открытия заявки'], row['Дата закрытия заявки']), axis=1) / 28800
df_z.head()

,ИД заявки,Вид оценки,Сегмент,подразделение эксперта,Цель рассмотрения заявки,Дата закрытия заявки,Дата открытия заявки,Назначение заявки,Автоматизация заявки,Классификатор Плышевского,Длительность,Рабочие дни
1556,141c007f6f64554345627d687f,"Рыночная, Контрактная",Средние,ЦООП (Ярославль),Закрыта по СЗ,2024-03-28 10:53:38.790,2022-07-28 08:50:18.782,Технические операции,Не автоматизирована,Технические операции,609,413.487500
1557,141c007f6f645543456d7f6f7e,"Кадастровая, Номинальная, Рыночная",Региональный госсектор,МЦК Санкт-Петербург,Закрыта по СЗ,2024-12-13 16:37:08.413,2022-08-02 10:20:13.068,Технические операции,Не автоматизирована,Технические операции,864,589.972917
1560,141c007f6f6455434664796d7a,"Рыночная, Номинальная",Средние,МЦК Санкт-Петербург,Закрыта по СЗ,2024-12-05 08:29:24.908,2022-08-16 13:04:09.547,Технические операции,Не автоматизирована,Технические операции,841,572.835417
1566,141c007f6f645543466178697f,"Рыночная, Номинальная",Средние,МЦК Санкт-Петербург,Закрыта по СЗ,2024-12-05 08:29:47.654,2022-08-31 16:05:59.229,Технические операции,Не автоматизирована,Технические операции,826,561.427083
1576,141c007f6f645542456c796c7e,"Кадастровая, Номинальная, Прочая, Рыночная",Средние,МЦК Ростов-на-Дону,Новый залог,2024-02-07 12:27:44.020,2023-07-12 18:45:29.453,Экспертиза,Не автоматизирована,ФЖН,209,142.312500
1577,141c007f6f64554246647c687e,Балансовая,Средние,ЦООП (Ярославль),Закрыта по СЗ,2024-10-16 07:31:09.266,2023-07-13 09:52:15.317,Технические операции,Не автоматизирована,Технические операции,460,314.016667
1590,141c007f6f6455424067766877,"Кадастровая, Балансовая",Крупнейшие,ЦООП (Ярославль),Новый залог,2024-02-27 11:50:35.754,2024-02-22 11:01:20.709,Экспертиза,Не автоматизирована,Экспертиза по нерыночной ст-ти,5,2.166667
1591,141c007f6f6455424060766f7a,"Рыночная, Прочая",Средние,ЦООП (Ярославль),Новый залог,2024-03-04 09:16:50.335,2024-03-04 09:14:53.167,Экспертиза,Не автоматизирована,ОО_Спецтехника,0,0.004167
1594,141c007f6f64554241657d687a,"Рыночная, Номинальная, Прочая",Средние,МЦК Москва,Переоценка,2024-01-16 17:45:15.497,2024-01-15 15:30:01.405,Экспертиза,Не автоматизирована,ФЖН,1,1.314583
1618,141c007f6f64554242667b6c78,"Рыночная, Номинальная, Балансовая, Прочая",Крупные,ЦООП (Ярославль),Последующий залог,2024-06-13 13:13:47.705,2023-10-23 10:04:10.898,Экспертиза,Не автоматизирована,Бессрочное СЗЗ,234,153.552083


##### Составьте алгоритм расчета рабочего времени без холдов

К сожалению с дальнейшим выполнением задания возникли технические проблемы, мой ноутбук выполняет скрипты выше часами(
Работу с корреляцией между данными можно увидеть в моем проекте 